In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import xgboost as xgb
plt.style.use('seaborn-bright')
#Read the dataset and print the top 5 elements of the dataset
train_data = pd.read_csv('updated_train_data.csv')
test_data = pd.read_csv('test_data.csv')

C:\Users\HAnu\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train_data['Month'] = train_data.date.astype(str).str[5:7].astype(float)
train_data['Year'] = train_data.date.astype(str).str[0:4].astype(float)
train_data['Day'] = train_data.date.astype(str).str[8:].astype(float)

train_data.drop(['date'], axis=1, inplace=True)
train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
train_data.head(5)

,hour,demand,Month,Year,Day
0,0,73.0,8.0,2018.0,18.0
1,1,73.0,8.0,2018.0,18.0
2,2,73.0,8.0,2018.0,18.0
3,3,73.0,8.0,2018.0,18.0
4,4,73.0,8.0,2018.0,18.0


In [3]:
y_train, X_train = train_data.loc[:,'demand'].values, train_data.loc[:,['Month', 'Year','Day','hour']].values
data_dmatrix = xgb.DMatrix(X_train,label=y_train)

In [4]:
test_data['Month'] = test_data.date.astype(str).str[5:7].astype(float)
test_data['Year'] = test_data.date.astype(str).str[0:4].astype(float)
test_data['Day'] = test_data.date.astype(str).str[8:].astype(float)

test_data.drop(['date'], axis=1, inplace=True)
test_data.replace([np.inf, -np.inf], np.nan, inplace=True)
test_data.head(5)

,hour,Month,Year,Day
0,0,3.0,2021.0,1.0
1,1,3.0,2021.0,1.0
2,2,3.0,2021.0,1.0
3,3,3.0,2021.0,1.0
4,5,3.0,2021.0,1.0


In [5]:
test_data=test_data.loc[:,['Month', 'Year','Day','hour']].values


In [6]:
reg_mod = xgb.XGBRegressor(
    n_estimators=1500,
    learning_rate=0.01,
    subsample=0.75,
    colsample_bytree=1, 
    max_depth=7,
    gamma=0,
)
reg_mod.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1500, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.75,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [7]:
predictions = reg_mod.predict(test_data)

In [8]:
predictions[0:10]

array([87.42794 , 63.137142, 54.673878, 60.324726, 51.04295 , 43.028687,
       64.52859 , 74.618004, 65.2937  , 71.98424 ], dtype=float32)

In [10]:
sub=pd.read_csv('submission_file.csv')
sub['demand']=predictions
sub['demand']=sub['demand'].astype('int')
sub.to_csv('Submission_XGBoost_V3.csv',index=False)